In [1]:
# Importing the required libraries
import os
from datasets import Dataset
from transformers import (AutoTokenizer, AutoModelForTokenClassification,
                          AutoConfig, Trainer, TrainingArguments, DataCollatorForTokenClassification)

from transformers import (BertTokenizerFast,
                          BertForTokenClassification,
                          Trainer,
                          TrainingArguments)
import torch
# Importing custom classes for loading and labeling the corpus
from utils.corpusprocessor import CorpusType
from utils.corpusprocessor import CorpusLoader
from utils.labeler import Labeler


2023-10-15 20:34:08.793695: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-15 20:34:08.793721: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Loading the corpus using the custom CorpusLoader class
corpus = CorpusLoader()
labeler = Labeler( tags=(1, 2),
                 regexes=(r'[^\S\r\n\v\f]', r'\u200c'),
                 chars=(" ", "‌"),
                 class_count=2,
                 )



In [3]:
data = corpus.load_bijan(CorpusType.whole_raw)
labeler.set_text(data, corpus_type=CorpusType.whole_raw)
chars, labels = labeler.labeler()
chars = chars[:3000]
labels = labels[:3000]

In [4]:
print(chars[:30])
print(labels[:30])
print('Character Length:',len(chars))
print(len(labels))

['#', 'ا', 'و', 'ل', 'ي', 'ن', 'س', 'ي', 'ا', 'ر', 'ه', 'خ', 'ا', 'ر', 'ج', 'ا', 'ز', 'م', 'ن', 'ظ', 'و', 'م', 'ه', 'ش', 'م', 'س', 'ي', 'د', 'ي', 'د']
[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
Character Length: 3000
3000


In [5]:
model_dir = "./Model2112/"
# pretrained_model = "HooshvareLab/bert-base-parsbert-uncased"
pretrained_model = "bert-base-multilingual-uncased"

In [6]:
def create_2d_list(input_list, ax1):
    return [input_list[i:i + ax1] for i in range(0, len(input_list), ax1)]

In [7]:

tokenizer = BertTokenizerFast.from_pretrained(pretrained_model)
data = [{"text": " ".join(chars), "labels": labels}]
tokenized_data = tokenizer([item['text'] for item in data], truncation=True, padding=True)
# Prepare the labels. Here I'm simply padding the labels list with -100s (the default ignore index in PyTorch)
labels = [item['labels'] + [-100] * (len(tokenized_data['input_ids'][i]) - len(item['labels'])) for i, item in enumerate(data)]

import numpy as np
print(np.array(tokenized_data['input_ids'][0]).shape)

(512,)


In [8]:
model = BertForTokenClassification.from_pretrained(pretrained_model,num_labels=3)
model

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [10]:
labels = create_2d_list(labels[0],512)
tokenized_data['input_ids'] =create_2d_list(tokenized_data['input_ids'][0],512)
import numpy as np
print(np.array(tokenized_data['input_ids']).shape)
print(np.array(labels).shape)
labels


(1, 512)
(1, 512)


[[1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  2,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  1,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  2,
  0,
  1,
  0,
  0,
  1,
  0,
  0,
  2,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  0,


In [21]:
# # Prepare everything for training
# train_dataset = torch.utils.data.Dataset()
# train_dataset.encodings = tokenized_data
# train_dataset.labels = labels


dataset = Dataset.from_dict({"input_ids": tokenized_data['input_ids'], "labels": labels})
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

ArrowInvalid: cannot mix list and non-list, non-null values

In [18]:

# Setting up training arguments for the Trainer class
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=2,
    max_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    warmup_steps=500,
    logging_dir=model_dir+'/logs',
    evaluation_strategy="no",
    save_strategy="no"
)

# Initializing a Trainer instance with the model, training arguments, dataset, tokenizer, and data collator
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=None,
    tokenizer=tokenizer,
    data_collator=data_collator
)



In [19]:
# Training the model on the dataset
trainer.train()


trainer.save_model(model_dir + "model/") 

ValueError: Expected input batch_size (512) to match target batch_size (3000).